# Worksheet 08

Name:  Emily Opresnick
UID: U91979014

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.2170032679562945, 6.0001139083436845, 4.747678661402947, 1.9111665505290634, 2.628111816961667, 5.167883363287036, 3.8002443294677306, 5.546512674669497, 3.7383466297893158, 5.347743565102469]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[8.771120075811293, 8.255333446039408, 9.637825860415253, 7.290977056163776, 8.382301576650793, 6.940705265377784, 8.456273459759858, 6.767258073975464, 7.608845309657503, 9.393175054613828]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[9.393175054613828, 5.347743565102469, 3.7383466297893158, 7.608845309657503, 6.767258073975464, 5.546512674669497, 3.8002443294677306, 5.167883363287036, 2.628111816961667, 8.456273459759858]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Priror probability (here it is 0.5), the mean and variance for the distribution

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2)/len(s1) + len(s2) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x: (x- mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.347743565102469, 3.7383466297893158, 5.546512674669497, 3.8002443294677306, 5.167883363287036, 2.628111816961667]
[9.393175054613828, 7.608845309657503, 6.767258073975464, 8.456273459759858]
P(S_1) = 0.6,  P(S_2) = 4.666666666666667
mean_1 = 4.371473729879619,  mean_2 = 8.056387974501662
var_1 = 1.1224252215499535,  var_2 = 0.9522646090403722


mean1 is far

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_s[0] )
    prob_s1_x.append(  pdf_i[1] * prob_s[1] /prob_s[1] )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  9.393175054613828
probability of observing that point if it came from cluster 0 =  1.6004314824425184e-05
probability of observing that point if it came from cluster 1 =  0.1563981147342194
point =  5.347743565102469
probability of observing that point if it came from cluster 0 =  0.2434864054899611
probability of observing that point if it came from cluster 1 =  0.0073327809001836235
point =  3.7383466297893158
probability of observing that point if it came from cluster 0 =  0.3031528665048224
probability of observing that point if it came from cluster 1 =  1.4362939674351082e-05
point =  7.608845309657503
probability of observing that point if it came from cluster 0 =  0.005550196034833126
probability of observing that point if it came from cluster 1 =  0.37513643580300204
point =  6.767258073975464
probability of observing that point if it came from cluster 0 =  0.036428131065417535
probability of observing that point if it came from cluster 1 =  0.16757100129446137
point =

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [10]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 4.666666666666667
mean_1 = 4.539782049950071,  mean_2 = 7.978518779414117
var_1 = 0.9876455155564183,  var_2 = 0.7758812798882179


mean1 is greater whereas mean2 is less

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [14]:

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)


k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_s[0] )
    prob_s1_x.append(  pdf_i[1] * prob_s[1] /prob_s[1] )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  9.393175054613828
probability of observing that point if it came from cluster 0 =  2.3043376325014357e-06
probability of observing that point if it came from cluster 1 =  0.09755145745295603
point =  5.347743565102469
probability of observing that point if it came from cluster 0 =  0.28905884607479176
probability of observing that point if it came from cluster 1 =  0.0016391421746048806
point =  3.7383466297893158
probability of observing that point if it came from cluster 0 =  0.29061926147266554
probability of observing that point if it came from cluster 1 =  1.6819175191212555e-07
point =  7.608845309657503
probability of observing that point if it came from cluster 0 =  0.003232035769786474
probability of observing that point if it came from cluster 1 =  0.4590078151706989
point =  6.767258073975464
probability of observing that point if it came from cluster 0 =  0.03175244933235791
probability of observing that point if it came from cluster 1 =  0.15201673767014445
point 

The probabilities of it beloning to a cluster became larger

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [13]:
assignments = [[], []]
for x in zip(data, prob_s0_x):
    p = x[1]
    d = x[0]
    if random.random() <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [3.8002443294677306, 5.167883363287036]
points in cluster 1 : [9.393175054613828, 5.347743565102469, 3.7383466297893158, 7.608845309657503, 6.767258073975464, 5.546512674669497, 2.628111816961667, 8.456273459759858]
